In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime
%matplotlib inline


In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [4]:
#Import data
def stock_data(ticker, start, end):
    try:
        data = yf.download(ticker,start, end, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return data

In [5]:
#Import S&P500
prices = stock_data(['SPY','GLD'], "2000-08-01", "2025-12-31")
prices.head(20)


[*********************100%***********************]  2 of 2 completed


Ticker,GLD,SPY
Date,,
2000-08-01,NaN,91.077019
2000-08-02,NaN,91.532066
2000-08-03,NaN,92.165085
2000-08-04,NaN,92.659592
2000-08-07,NaN,93.767380
2000-08-08,NaN,94.123520
2000-08-09,NaN,93.332230
2000-08-10,NaN,92.877228
2000-08-11,NaN,93.312447


In [6]:
prices.dropna()

Ticker,GLD,SPY
Date,,
2004-11-18,44.380001,80.169052
2004-11-19,44.779999,79.277847
2004-11-22,44.950001,79.655922
2004-11-23,44.750000,79.777451
2004-11-24,45.049999,79.966492
...,...,...
2025-12-23,413.640015,687.960022
2025-12-24,411.929993,690.380005
2025-12-26,416.739990,690.309998


In [7]:
class RotationStrategy:
    
    def __init__(self,
                prices =pd.DataFrame):
        """ SPY and GLD daily prices data  """
        self.prices =prices
        
    
    def rotation_strategy(self, ma_window=6):
        """calculating the strategy"""

        #Resampling to get the price of the last business day of the month
        monthly_prices = prices.resample("ME").last()
        monthly_prices.dropna(inplace=True)
        
        #Track the ratio of Gold prices to S&P500
        monthly_prices["price_ratio"] = monthly_prices['GLD']/monthly_prices['SPY']

        #Moving Average of the ratio
        monthly_prices['ratio_moving_avg']= monthly_prices['price_ratio'].rolling(window= ma_window).mean()
        monthly_prices.dropna(inplace=True)

        #Generating signal
        signal = (monthly_prices['price_ratio'] < monthly_prices['ratio_moving_avg']).astype(int)
        signal = signal.shift(1).dropna()

        #Calculating monthly returns
        monthly_returns = monthly_prices.pct_change().loc[signal.index]

        #Calculating return startegy and equity curve
        strategy_returns = (signal*monthly_returns['SPY'] + (1-signal) * monthly_returns['GLD'])
        equity_curve = (1+strategy_returns).cumprod()
        
        #calculating returns for buy and hold strategy
        gold_bnh = (1+monthly_returns['GLD']).cumprod()
        spy_bnh = (1+monthly_returns['SPY']).cumprod()

        #Plotting the curves
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve,
                         mode='lines', name = 'Equity Curve'))
        fig.add_trace(go.Scatter(x=gold_bnh.index, y = gold_bnh,
                         mode='lines', name = 'Gold Buy and Hold'))
        fig.add_trace(go.Scatter(x=spy_bnh.index, y = spy_bnh,
                         mode='lines', name = 'S&P Buy and hold'))

        fig.update_layout(title='Equity Curve', 
                  xaxis_title='Date', yaxis_title ='Performance',
                  template="plotly_white",
            hovermode="x unified")
        fig.show()
        



In [8]:
strategy = RotationStrategy(prices)

In [9]:
strategy.rotation_strategy(6)